# Init

In [1]:
from IPython.display import display
from typing import List,Dict
import os
import sys
import psycopg2
from psycopg2.extras import NamedTupleCursor
os.chdir('/home/rchuzh/programming/image_labelling_shrdc')
from pathlib import Path
Path.cwd()
SRC = Path('/home/rchuzh/programming/image_labelling_shrdc/src')  # ROOT folder -> ./src
LIB_PATH = SRC / "lib"

if str(LIB_PATH) not in sys.path:
    sys.path.insert(0, str(LIB_PATH))  # ./lib
else:
    pass

In [2]:
from data_manager.database_manager import db_fetchone,db_no_fetch,db_fetchall
from core.utils.log import log_info,log_error
dsn = "host=localhost port=5432 dbname=eye user=shrdc password=shrdc"
def init_connection(dsn=None, connection_factory=None, cursor_factory=None, **kwargs):
    """ Connect to the PostgreSQL database server """

    try:
        # read connection parameters
        # params = config()

        # connect to the PostgreSQL server
        log_info('Connecting to the PostgreSQL database...')
        if kwargs:
            conn = psycopg2.connect(**kwargs)

        else:
            conn = psycopg2.connect(dsn, connection_factory, cursor_factory)

        # create a cursor
        with conn:
            with conn.cursor(cursor_factory=NamedTupleCursor) as cur:

                # execute a statement
                cur.execute('SELECT version();')
                conn.commit()

                # display the PostgreSQL database server version
                db_version = cur.fetchone().version
                log_info(f"PostgreSQL database version: {db_version}")
                log_info(f"PostgreSQL connection status: {conn.info.status}")
                log_info(
                    f"You are connected to database '{conn.info.dbname}' as user '{conn.info.user}' on host '{conn.info.host}' at port '{conn.info.port}'.")
        return conn
    except (Exception, psycopg2.DatabaseError) as error:
        log_error(error)
        conn = None


In [3]:
conn=init_connection(dsn)

2021-07-28 09:51:50.290 INFO    root: Connecting to the PostgreSQL database...
2021-07-28 09:51:50.315 INFO    root: PostgreSQL database version: PostgreSQL 13.3 (Ubuntu 13.3-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit
2021-07-28 09:51:50.317 INFO    root: PostgreSQL connection status: 0
2021-07-28 09:51:50.319 INFO    root: You are connected to database 'eye' as user 'shrdc' on host 'localhost' at port '5432'.


# 28/7/2021

In [16]:
from enum import IntEnum
class DeploymentType(IntEnum):
    Image_Classification = 1
    OD = 2
    Instance = 3
    Semantic = 4

    def __str__(self):
        return self.name

    @classmethod
    def from_string(cls, s):
        try:
            return DeploymentType[s]
        except KeyError:
            raise ValueError()

In [17]:
format=DeploymentType.from_string('OD')
format == DeploymentType.OD

True

In [65]:
query_dt_sql="""SELECT
                    p.project_path,
                    t.name
                FROM
                    public.models m
                    INNER JOIN public.training t ON m.training_id = t.id
                    INNER JOIN public.project p ON t.project_id = p.id
                WHERE
                    m.id = 2;
"""
return_all=db_fetchone(query_dt_sql,conn)
return_all

[Record(project_path='/home/rchuzh/.local/share/integrated-vision-inspection-system/app_media/project/my-sixth-project', name='not so serious training')]

In [62]:
for i in return_all:
    display(i)

Record(ID=1, Name='[TF] SSD MobileNet V2 FPNLite 320x320', Framework='TensorFlow', Deployment_Type='Object Detection with Bounding Boxes', Model_Path='./pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8')

Record(ID=2, Name='[TF] SSD ResNet50 V1 FPN 640x640 (RetinaNet50)', Framework='TensorFlow', Deployment_Type='Object Detection with Bounding Boxes', Model_Path='./pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8')

In [55]:
y=[i.Model_Path for i in return_all if i.Name=='[TF] SSD MobileNet V2 FPNLite 320x320'][0]
y

'./pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'

In [56]:
model_list=[model.Name for model in return_all]

In [61]:
model_selected='[TF] SSD MobileNet V2 FPNLite 320x320'
model_id=return_all[model_list.index("[TF] SSD MobileNet V2 FPNLite 320x320")].ID
model_id

1